<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/gsrc/evaluating_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install torch
!pip install bert_score
!pip install evaluate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.2 MB/s eta 0:00:00


In [54]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset, load_metric
import torch
import pandas as pd

# Load model and tokenizer
checkpoint = "HamdanXI/t5_small_gloss_merged_dataset_adj_adv"
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5Tokenizer.from_pretrained(checkpoint)

# Load dataset
def load_text_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
    return pd.DataFrame(lines, columns=['text'])

def prepare_data(gloss_path, text_path):
  asl_df = load_text_file(gloss_path)
  text_df = load_text_file(text_path)
  asl_df.columns = ['gloss']
  df = pd.concat([text_df, asl_df], axis=1)
  # Strip spaces from the start and end of each sentence in the 'text' column
  df['text'] = df['text'].apply(lambda x: x.strip() if pd.notnull(x) else '')
  # Strip spaces from the start and end of each sentence in the 'gloss' column
  df['gloss'] = df['gloss'].apply(lambda x: x.strip() if pd.notnull(x) else '')
  return df

# dataset = load_dataset("aslg_pc12")
dataset = prepare_data("gloss.txt", "text.txt")


# Tokenize the evaluation dataset
input_texts = dataset["gloss"]
target_texts = dataset["text"]

input_texts_1 = input_texts.tolist()
target_texts_1 = target_texts.tolist()

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.63k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [55]:
max_token_length_gloss = max(len(tokenizer.encode(item)) for item in dataset["gloss"])
max_token_length_text = max(len(tokenizer.encode(item)) for item in dataset["text"])

print(f"Max token length for gloss: {max_token_length_gloss}")
print(f"Max token length for text: {max_token_length_text}")

Max token length for gloss: 49
Max token length for text: 26


In [56]:
inputs = tokenizer(input_texts_1, return_tensors="pt", padding=True, truncation=True, max_length=60)
labels = tokenizer(target_texts_1, return_tensors="pt", padding=True, truncation=True, max_length=60).input_ids

# Make predictions
model.eval()

with torch.no_grad():
    outputs = model.generate(inputs.input_ids)
decoded_predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Compute BERTScore
metric = load_metric("bertscore")
score = metric.compute(predictions=decoded_predictions, references=target_texts, lang="en")

# Compute the mean values
mean_precision = sum(score['precision']) / len(score['precision'])
mean_recall = sum(score['recall']) / len(score['recall'])
mean_f1 = sum(score['f1']) / len(score['f1'])

# similarly, you can compute mean_recall and mean_f1 if needed

print(f"Mean Precision: {mean_precision}")
print(f"Mean Recall: {mean_recall}")
print(f"Mean F1: {mean_f1}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You 

Mean Precision: 0.8638948995178029
Mean Recall: 0.8896280562473555
Mean F1: 0.8762297670719987


In [57]:
import evaluate

metric = evaluate.load("sacrebleu")

In [58]:
outputs = []  # List to store the outputs

for text in input_texts:
    # Encode the input text
    input_ids = tokenizer.encode(text, return_tensors="pt")

    # Generate output sequences
    output_sequences = model.generate(input_ids)

    # Decode the output sequences to strings
    output_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # Append the output text to the outputs list
    outputs.append(output_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [59]:
results = metric.compute(predictions=outputs, references=target_texts_1)

print(results)

{'score': 5.107833757972903, 'counts': [279, 71, 23, 7], 'totals': [1014, 896, 779, 662], 'precisions': [27.514792899408285, 7.924107142857143, 2.952503209242619, 1.0574018126888218], 'bp': 1.0, 'sys_len': 1014, 'ref_len': 797}
